In [4]:
import warnings
from ftfy import fix_text
from geopy.distance import geodesic
import pandas as pd
from rapidfuzz import process, fuzz
import carga_data as cd

In [2]:
warnings.filterwarnings('ignore')

---
---
# Yelp


## Carga de dataset

In [5]:
# Path de los archivos no-procesados (formato parquet)
path_data = '../../data/raw'
yelp = cd.cargar_dataset_yelp(path_data)
yelp.keys()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/y-business.parquet'

In [4]:
df_business, df_checkin, df_tip, df_review, df_user = yelp.values()

## Transformaciones

### Tabla `business`

In [5]:
# Convertir columna 'categories' a tipo list
df_business['categories'] = df_business['categories'].apply(lambda x: x.strip().replace(', ',',').split(',') if x else [])

Filtro generales, basado en enfoque del proyecto (restaurantes dentro de una area geografica)

In [6]:
## Filtros principales del dataset
#  Toda otra tabla del dataset se filtra en base a los 
#  `business_id` que se encuentran en el `df_business` procesado.

# Filtrar donde `is_open == 1`
df_business = df_business[df_business['is_open'] == 1]
# Descartar col `is_open`
df_business.drop('is_open', axis=1, inplace=True)

# Filtrar por categoría 'Restaurants'
filtro_categoria = 'Restaurants'
df_business = df_business[df_business['categories'].apply(lambda x: filtro_categoria in x)]

# Filtrar por estados de interes
filtro_estado = ['DE', 'NJ', 'PA']
df_business = df_business[df_business['state'].isin(filtro_estado)]

df_business.shape

(10203, 13)

Filtro por coordinadas

In [ ]:
def es_dentro_de_area(latitude, longitude, punto_centro, radio=200) -> bool:
    """Devuelve valor boolean, True si `latitude` y `longitude` de entrada estan
    dentro del area definido por `punto_centro` y `radio. De lo contrario,
    devuelve False.
    """
    # Calcular la distancia desde el punto central `punto_centro`
    distancia = geodesic((latitude, longitude), punto_centro).miles
    # Check if the distance is within the specified radius
    return distancia <= radio

In [ ]:
# Coordinadas de Philadelphia, PA
coord_philadelphia = (39.9526, -75.1652)

# Crear nuava columna con 
df_business['es_dentro_de_area'] = df_business.apply(lambda x: es_dentro_de_area(x['latitude'], x['longitude'], coord_philadelphia), axis=1)

df_business = df_business[df_business['es_dentro_de_area']]

Nuevas columnas y tablas

In [7]:
# Crear un dataframe nuevo para los atributos
df_attributes = df_business[['business_id', 'name', 'attributes']]
# Descartar col `attributes`
df_business.drop('attributes', axis=1, inplace=True)

In [8]:
# Crear nueva columna de coordinadas
df_business['coordinates'] = df_business['latitude'].astype(str) + ',' + df_business['longitude'].astype(str)

### Tabla `checkin`

In [10]:
# Filtrar por `business_id` en el `df_business` procesado
filtro_id = df_business['business_id']
df_checkin = df_checkin[df_checkin['business_id'].isin(filtro_id)]
df_checkin.shape

(10059, 2)

In [ ]:
# Convertir col `date` a tipo list
df_checkin['date'] = df_checkin['date'].apply(lambda x: x.strip().replace(', ',',').split(',') if x else [])
# Convertir items en listas de col `date` a tipo datetime 
df_checkin['date'] = df_checkin['date'].apply(lambda x: [pd.to_datetime(string, format='mixed') for string in x] if x else x) 

### Tabla `tip`

In [ ]:
# Filtrar por `'business_id'` en el `df_business` procesado
filtro_id = df_business["business_id"]
df_tip = df_tip[df_tip["business_id"].isin(filtro_id)]
df_tip.shape

### Tabla `review`

In [11]:
# Filtrar por `'business_id'` en el `df_business` procesado
filtro_id = df_business["business_id"]
df_review = df_review[df_review["business_id"].isin(filtro_id)]
df_review.shape

(1138498, 9)

In [12]:
## Transformaciones

# Convertir col `date` a tipo datetime
df_review['date'] = pd.to_datetime(df_review['date'], format='mixed')

# Corrigir errores de decoding de texto, columna `text`
df_review['text'] = df_review['text'].apply(fix_text)

# Agregar col `name`, desde `df_business`
df_review = pd.merge(df_review, df_business[['business_id', 'name']], on='business_id', how='left')
# Fijar nueva columna `name` como la primer columna del df 
nom_col = df_review.pop('name')
df_review.insert(0, 'name', nom_col)

### Tabla `user`

In [ ]:
# Filtrar por `'user_id'` en el `df_review` procesado
filtro_id = df_review['user_id']
df_user = df_user[df_user['user_id'].isin(filtro_id)]
df_user.shape

In [ ]:
## Transformaciones

# Convertir col `yelping_since` a tipo datetime
df_user['yelping_since'] = pd.to_datetime(df_user['yelping_since'], format='mixed')

# Convertir columna 'friends' a tipo list
df_user['friends'] = df_user['friends'].apply(lambda x: x.replace(', ',',').split(',') if x != 'None' else [])

## Exportaciones data procesada Yelp

In [13]:
## Almacenar dataframes en formato parquet

# Path de los archivos procesados (formato parquet)
path_data_clean = '../../data/clean'

df_business.to_parquet(f'{path_data_clean}/y_business_CLEAN.parquet')
df_attributes.to_parquet(f'{path_data_clean}/y_attributes_CLEAN.parquet')
df_checkin.to_parquet(f'{path_data_clean}/y_checkin_CLEAN.parquet')
df_tip.to_parquet(f'{path_data_clean}/y_tip_CLEAN.parquet')
df_review.to_parquet(f'{path_data_clean}/y_review_CLEAN.parquet')
df_user.to_parquet(f'{path_data_clean}/y_user_CLEAN.parquet')

In [ ]:
# Liberar memoria, eliminando las variables de dataframes
del yelp
del df_business
del df_checkin
del df_tip
del df_review
del df_user

---
---
# Google

## Carga de dataset

In [ ]:
# Path de los archivos no-procesados (formato parquet)
path_data = '../../data/raw'
goog = cd.cargar_dataset_google(path_data)
goog.keys()

In [ ]:
df_sitios, df_review = goog.values()

Tambien se carga la tabla `business` del dataset de Yelp para facilitar el filtrado de `df_sitios`

In [ ]:
# Path de los archivos procesados (formato parquet)
path_data_clean = '../../data/clean'
df_yelp_business = pd.read_parquet(f'{path_data_clean}/y_business_CLEAN.parquet')

## Transformaciones

### Filtrado de validacion cruzada con Yelp

In [ ]:
# Funcion que implementa busqueda 'fuzzy' en textos
def fuzzy_match(x, match_to, threshold=90):
    match, score, _ = process.extractOne(x, match_to, scorer=fuzz.WRatio)
    return match if score >= threshold else None

## Exportaciones data procesada Google

In [ ]:
## Almacenar dataframes en formato parquet

# Path de los archivos procesados (formato parquet)
path_data = '../../data/clean'

df_sitios.to_parquet(f'{path_data}/g_sitios_CLEAN.parquet')
df_review.to_parquet(f'{path_data}/g_reviews_CLEAN.parquet')

In [ ]:
# Liberar memoria, eliminando las variables de dataframes
del goog
del df_sitios
del df_review